In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

d:\Documents\SelfProject\IndoRecipe2


In [2]:
import pandas as pd

In [3]:
from datasets import Dataset

In [4]:
from idrecibrew2.data.indonlg_tokenizer.tokenizer import IndoNLGTokenizer
from transformers import DataCollatorForSeq2Seq

In [5]:
tokenizer = IndoNLGTokenizer.from_pretrained("indobenchmark/indobart-v2")

In [6]:
df = pd.read_csv("data/processed/test.csv")

In [7]:
data = Dataset.from_pandas(df.iloc[0:10])

In [8]:
tokenizer.tokenize("Testing yah")

['▁', '<0x54>', 'esting', '▁yah']

In [9]:
column_source = 'src'
column_target = 'tgt'

In [10]:
max_input_length = 128
max_target_length = 256

def preprocess_function(examples):
    inputs = examples['src']
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["tgt"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
def process_data(example_data, idx):
    tokenized_input = {}
    tokenized_input.update({key: dict(tokenizer(example_data[key]))['input_ids'] for key in [column_source, column_target]})
    return tokenized_input

In [12]:
hasil = data.map(preprocess_function, with_indices=False, batched=True)

100%|██████████| 1/1 [00:00<00:00, 63.82ba/s]


In [13]:
hasil

Dataset({
    features: ['attention_mask', 'basic_ingredient', 'input_ids', 'labels', 'no', 'src', 'tgt'],
    num_rows: 10
})

In [14]:
hasil = hasil.remove_columns(['no', 'basic_ingredient', 'src', 'tgt'])

In [15]:
hasil

Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 10
})

In [16]:
from torch.utils.data import DataLoader

In [17]:
collator = DataCollatorForSeq2Seq(tokenizer, padding=True)

In [18]:
coba = DataLoader(hasil, 4, collate_fn = collator)

In [19]:
next(iter(coba))

{'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'input_ids': tensor([[    0, 25270, 12338,  3353,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   486,  5957,  2681,   306, 39974, 16576,   299, 33716, 12338,
          7727,   327,   244,   163,   139,   178,   244,   163,   139,   173,
         39965,     2],
        [    0,  3569,  3793, 12338, 19340,  4046, 24919,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   771,   459, 12338,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1, 

In [20]:
tokenizer.vocab_size

C:\Users\62852\miniconda3\envs\idrecibrew2\lib\importlib\__init__.py:169: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  _bootstrap._exec(spec, module)


40004